In [1]:
import os
from datetime import datetime

import pandas as pd
import numpy as np

from pyomo.environ import *

from data_cleaning import site_occ_dict, blockgroup_pop_dict, bg_ces_dict, dist_to_site_df, dist_to_site_dict

#dist_to_site_df = pd.read_csv('data/distmatrix_contracosta.csv')
#dist_to_site_df.set_index('Unnamed: 0', inplace = True)
#dist_to_site_df.index.name = None

### Define Model
_Create model and initial indices_

In [2]:
model = ConcreteModel()

model.init_bgs = Set(initialize = dist_to_site_df.index)
model.init_sites = Set(initialize = dist_to_site_df.columns)

_Create set of blockgroup, site pairs within five miles of each other_

In [3]:
def filter_to_nearby_sites(model, bg, site):
    return not np.isnan(dist_to_site_df.loc[bg, site])

model.idx_bg_site_pairs = Set(initialize = model.init_bgs*model.init_sites, filter = filter_to_nearby_sites)

_Create new index of sites and bgs only including those that belong in >5 mile pair_

In [4]:
filtered_bgs = []
filtered_sites = []

for pair in model.idx_bg_site_pairs:
    if pair[0] not in filtered_bgs:
        filtered_bgs.append(pair[0])
    if pair[1] not in filtered_sites:
        filtered_sites.append(pair[1])
        
model.idx_bgs = Set(initialize = filtered_bgs)
model.idx_sites = Set(initialize = filtered_sites)

### Define Parameters

_Number of people per blockgroup_

In [5]:
def get_blockgroup_pop(model, bg):
    return(blockgroup_pop_dict[bg])

model.param_bg_pop = Param(model.idx_bgs, initialize = get_blockgroup_pop)

_Distance from blockgroup to site_

In [6]:
def get_bg_site_dist(model, bg, site):
    return(dist_to_site_df.loc[bg, site])

model.param_bg_site_dist = Param(model.idx_bg_site_pairs, initialize = get_bg_site_dist)

_Max capacity per site_

In [7]:
def get_site_capacity(model, site):
    return(site_occ_dict[site])

model.param_site_cap = Param(model.idx_sites, initialize = get_site_capacity)

_Sites within range for each blockgroup_

In [8]:
bg_sites_in_range = {bg: [] for bg in model.idx_bgs}

for bg in bg_sites_in_range:
    for pair in model.idx_bg_site_pairs:
        if pair[0] == bg:
            bg_sites_in_range[bg].append(pair[1])

bg_with_no_hub = [key for key in bg_sites_in_range if len(bg_sites_in_range[key]) == 0]

for bg in bg_with_no_hub:
    del bg_sites_in_range[bg]
    
model.param_bg_sites_in_range = Param(model.idx_bgs, within=Any, initialize=bg_sites_in_range)

_Blockgroups within range for each site_

In [9]:
site_bgs_in_range = {site: [] for site in model.idx_sites}

for site in site_bgs_in_range:
    for pair in model.idx_bg_site_pairs:
        if pair[1] == site:
            site_bgs_in_range[site].append(pair[0])

site_with_no_hub = [key for key in site_bgs_in_range if len(site_bgs_in_range[key]) == 0]

for site in site_with_no_hub:
    del site_bgs_in_range[site]
    
model.param_site_bgs_in_range = Param(model.idx_sites, within=Any, initialize=site_bgs_in_range)

#### _Blockgroup climate and social characteristics_
_Calenviroscreen score_

In [10]:
def get_ces_score(model, bg):
    return(bg_ces_dict[bg])

model.param_bg_vuln_ces = Param(model.idx_bgs, within=Any, default = 0, initialize=get_ces_score)

### Define Variables

_Is this site a site?_

In [11]:
model.var_hub_yn = Var(model.idx_sites, initialize = 1, within = Binary)

_What prop of this blockgroup is served at this site?_

In [12]:
model.var_prop_bg_at_site = Var(model.idx_bg_site_pairs, initialize = 1.0, bounds = (0.0, 1.0))

### Define Objective

In [13]:
agg_dist = sum(model.param_bg_pop[bg]*model.var_prop_bg_at_site[bg, site]*model.param_bg_site_dist[bg, site] 
               for bg, site in model.idx_bg_site_pairs)

model.obj_min_agg_dist = Objective(expr = agg_dist, sense = minimize)

# _N Site Here_
### Define Constraints

_Construct set number of sites_

In [94]:
n_sites = sum(model.var_hub_yn[site] for site in model.idx_sites)

model.con_max_sites = Constraint(expr = (1, n_sites, 60))

    (type=<class 'pyomo.core.base.constraint.ScalarConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


_Do not let anyone go to a site if it is not a hub_

In [95]:
def serve_only_at_hubs(model, bg, site):
    return(model.var_prop_bg_at_site[bg, site] <= model.var_hub_yn[site])

model.con_serve_only_at_hubs = Constraint(model.idx_bg_site_pairs, rule = serve_only_at_hubs)

    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


_Serve a minimum proportion of area population_

In [96]:
pop_area_tot = sum(model.param_bg_pop[bg] for bg in model.idx_bgs)
pop_service_tot = sum(model.param_bg_pop[bg]*model.var_prop_bg_at_site[bg, site] for bg, site in model.idx_bg_site_pairs)

model.con_min_service_pop = Constraint(expr = pop_service_tot >= 0.05*pop_area_tot)

    (type=<class 'pyomo.core.base.constraint.ScalarConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


_Do not serve more than 100% of a blockgroup's demand_

In [97]:
def serve_only_demand(model, bg):
    bg_tot_pop = model.param_bg_pop[bg]
    bg_tot_served = sum(model.param_bg_pop[bg]*model.var_prop_bg_at_site[bg, site] for site in model.param_bg_sites_in_range[bg])
    
    return((0, bg_tot_served, bg_tot_pop))

model.con_serve_only_demand = Constraint(model.idx_bgs, rule = serve_only_demand)

    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


_Do not send more people to hub than it can fit_
* Debug note: when if statement written as ==1 (as opposed to !=1) and lowerbound added, (e.g. 0.75*site_tot_cap, site_tot_served, 1.0*site_tot_cap), serve_only_at_hubs stops working properly and many  hubs are added

In [98]:
def serve_max_capacity(model, site):
    site_tot_cap = model.param_site_cap[site]
    site_tot_served = sum(model.param_bg_pop[bg]*model.var_prop_bg_at_site[bg, site] for bg in model.param_site_bgs_in_range[site])
    
    if model.var_hub_yn[site].value != 1:
        return((0, site_tot_served, 0))
    else:      
        return((0, site_tot_served, 5*site_tot_cap))

model.con_serve_max_capacity = Constraint(model.idx_sites, rule = serve_max_capacity)

    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


# _CES HERE_
#### _Additional climate and social-based constraints_

In [99]:
def serve_ces_pops(model, bg):
    bg_ces_score = model.param_bg_vuln_ces[bg]
    
    bg_tot_prop_served = sum(model.var_prop_bg_at_site[bg, site] for site in model.param_bg_sites_in_range[bg])
    
    if bg_ces_score >= 0.75:
        return((0.95, bg_tot_prop_served, 1))
    else:
        return((0, bg_tot_prop_served, 1))

model.con_serve_ces_pops = Constraint(model.idx_bgs, rule = serve_ces_pops)

    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


### Set up Solver

In [100]:
import pyomo.opt as pyopt

In [101]:
result_folder = 'results/p-med/'
result_time  = datetime.now().strftime('%Y-%m-%d_%H%M')
result_path = result_folder + result_time

result = pyopt.SolverFactory('gurobi').solve(model)

print(result)


Problem: 
- Name: x24130
  Lower bound: 206729.95649948396
  Upper bound: 206729.95649948396
  Number of objectives: 1
  Number of constraints: 26673
  Number of variables: 24130
  Number of binary variables: 558
  Number of integer variables: 558
  Number of continuous variables: 23572
  Number of nonzeros: 190890
  Sense: minimize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 0.058506011962890625
  Error rc: 0
  Time: 0.5531003475189209
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [102]:
result_path_file =  result_path + '.mps'

model.write(result_path_file)

('results/p-med/2022-03-15_1048.mps', 1503251383104)

In [103]:
result_site = [site for site in model.idx_sites]
result_hub_yn = [model.var_hub_yn[site].value for site in model.idx_sites]

result_hub_yn_df = pd.DataFrame({'site_id': result_site, 'site_yn':result_hub_yn})

result_hub_yn_df = result_hub_yn_df.loc[result_hub_yn_df['site_yn'] > 0]

result_hub_yn_path_file = result_path + '_hub_yn.csv'
result_hub_yn_df.to_csv(result_hub_yn_path_file)

result_hub_yn_df.shape

(20, 2)

In [104]:
prop_bg_at_site_df = dist_to_site_df.copy()
prop_bg_at_site_df.loc[:] = np.NaN

for key in model.var_prop_bg_at_site.keys():
    prop_bg_at_site_df.loc[key] = model.var_prop_bg_at_site[key].value
    
prop_bg_at_site_df_notnull = prop_bg_at_site_df[prop_bg_at_site_df.notnull()]

result_prop_served_dict = {}

for key in model.var_prop_bg_at_site.keys():
    if prop_bg_at_site_df.loc[key]:
        result_prop_served_dict[key] = model.var_prop_bg_at_site[key].value

In [105]:
result_prop_served_hubs = set([key[1] for key in result_prop_served_dict.keys()])
result_prop_served_bgs = set([key[0] for key in result_prop_served_dict.keys()])

result_prop_served_df = pd.DataFrame(index = result_prop_served_bgs, columns = result_prop_served_hubs)

for bg, hub in result_prop_served_dict.keys():
    result_prop_served_df.loc[bg, hub] = result_prop_served_dict[bg, hub]
    
result_prop_served_df.fillna(0, inplace = True)

result_prop_served_path = result_path + '_prop_served.csv'
result_prop_served_df.to_csv(result_prop_served_path)

In [106]:
result_dist_traveled_df = result_prop_served_df.copy()

for bg, hub in result_prop_served_dict.keys():
        
    result_dist_traveled_df.loc[bg, hub] = dist_to_site_dict[bg, hub]*result_prop_served_df.loc[bg, hub]*blockgroup_pop_dict[bg]
    
result_dist_traveled_path = result_path + '_dist_traveled.csv'
result_dist_traveled_df.to_csv(result_dist_traveled_path)

print('Agg dist:', str(result_dist_traveled_df.sum().sum()))

Agg dist: 206729.95649948396


In [107]:
result_pop_df = result_prop_served_df.copy()

for bg, hub in result_prop_served_dict.keys():
        
    result_pop_df.loc[bg, hub] = result_prop_served_df.loc[bg, hub]*blockgroup_pop_dict[bg]
    
result_pop_path = result_path + '_pop.csv'
result_pop_df.to_csv(result_pop_path)

print('Agg pop:', str(result_pop_df.sum().sum()))

Agg pop: 194660.7


In [108]:
result_ces_df = result_prop_served_df.copy()

for bg, hub in result_prop_served_dict.keys():
        
    result_ces_df.loc[bg, hub] = bg_ces_dict[bg]*result_prop_served_df.loc[bg, hub]*blockgroup_pop_dict[bg]
    
result_ces_path = result_path + '_ces.csv'

result_ces_df.to_csv(result_ces_path)
result_ces_df.sum().sum()/result_pop_df.sum().sum()

0.8446934692005111

In [109]:
#site_occ_dict, blockgroup_pop_dict, bg_ces_dict, dist_to_site_df, dist_to_site_dict